<a href="https://colab.research.google.com/github/TirendazAcademy/Deep-Learning-with-TensorFlow/blob/main/Pratical-Keras-Tutorials/Keras_Custom_Layer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
from tensorflow import keras

# Custom Layer with Subclassing API

In [2]:
class Linear(keras.layers.Layer):
  def __init__(self, units=32, input_dim = 32):
    super(Linear,self).__init__()

    w_init= tf.random_normal_initializer()
    self.w=tf.Variable(
        initial_value=w_init(shape=(input_dim,units), dtype="float32"),
        trainable=True
    )
    b_init = tf.zeros_initializer()
    self.b = tf.Variable(
        initial_value=b_init(shape=(units,), dtype="float32"),
        trainable=True
    )
  def call(self,inputs):
    return tf.matmul(inputs, self.w) + self.b

In [3]:
x = tf.ones((2,2))
x

<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[1., 1.],
       [1., 1.]], dtype=float32)>

In [4]:
linear_layer = Linear(4,2)
y = linear_layer(x)
y

<tf.Tensor: shape=(2, 4), dtype=float32, numpy=
array([[-0.00113926, -0.15292075,  0.10305747,  0.03060768],
       [-0.00113926, -0.15292075,  0.10305747,  0.03060768]],
      dtype=float32)>

# Custom Layer using add_weight

In [5]:
from tensorflow.python.ops.init_ops_v2 import Initializer

class Linear(keras.layers.Layer):
  def __init__(self, units=32, input_dim = 32):
    super(Linear,self).__init__()
    self.w = self.add_weight(shape=(input_dim, units),
                             initializer="random_normal",
                             trainable = True)
    self.b = self.add_weight(shape=(units,),
                             initializer = "zeros",
                             trainable = True)
  def call(self,inputs):
    return tf.matmul(inputs, self.w) + self.b

In [6]:
x = tf.ones((2,2))
linear_layer = Linear(4,2)
y = linear_layer(x)

In [7]:
y

<tf.Tensor: shape=(2, 4), dtype=float32, numpy=
array([[ 0.0511761 ,  0.04031583,  0.05865897, -0.16737346],
       [ 0.0511761 ,  0.04031583,  0.05865897, -0.16737346]],
      dtype=float32)>

# Custom Layer without the shape of the inputs

In [8]:
class Linear(keras.layers.Layer):
  def __init__(self, units=32):
    super(Linear,self).__init__()
    self.units = units
  def build(self,input_shape):
    self.w = self.add_weight(shape=(input_shape[-1],self.units),
                             initializer = "random_normal",
                             trainable = True)
    self.b = self.add_weight(shape = (self.units,),
                             initializer = "random_normal",
                             trainable = True)
  def call(self, inputs):
    return tf.matmul(inputs, self.w) + self.b

In [9]:
linear_layer = Linear(32)
y = linear_layer(x)
y

<tf.Tensor: shape=(2, 32), dtype=float32, numpy=
array([[ 0.04310997, -0.0517299 ,  0.18074639,  0.07049532, -0.05648559,
         0.00300806,  0.00553202,  0.00923574, -0.11928318, -0.09789699,
         0.08400971,  0.00142121, -0.06042321, -0.01848508,  0.11042443,
        -0.00495854,  0.02891158, -0.02972268,  0.10572973, -0.07782042,
         0.01720538,  0.04416382,  0.12230952, -0.17262745, -0.03663931,
         0.05722104,  0.04017566, -0.03489885,  0.02452268,  0.01209868,
        -0.05540178, -0.05333155],
       [ 0.04310997, -0.0517299 ,  0.18074639,  0.07049532, -0.05648559,
         0.00300806,  0.00553202,  0.00923574, -0.11928318, -0.09789699,
         0.08400971,  0.00142121, -0.06042321, -0.01848508,  0.11042443,
        -0.00495854,  0.02891158, -0.02972268,  0.10572973, -0.07782042,
         0.01720538,  0.04416382,  0.12230952, -0.17262745, -0.03663931,
         0.05722104,  0.04017566, -0.03489885,  0.02452268,  0.01209868,
        -0.05540178, -0.05333155]], dtyp

# Layers are recursively composable

In [10]:
class MLPBlock(keras.layers.Layer):
  def __init__(self):
    super(MLPBlock, self).__init__()
    self.linear_1 = Linear(32)
    self.linear_2 = Linear(32)
    self.linear_3 = Linear(1)
  def call(self, inputs):
    x = self.linear_1(inputs)
    x = tf.nn.relu(x)
    x = self.linear_2(x)
    x = tf.nn.relu(x)
    return self.linear_3(x)

In [11]:
mlp = MLPBlock()
y = mlp(tf.ones(shape=(3,64)))

In [12]:
mlp.weights

[<Variable path=mlp_block/linear_3/variable_4, shape=(64, 32), dtype=float32, value=[[ 0.01439771 -0.03113151 -0.04914447 ... -0.09509321 -0.01546378
    0.04873939]
  [ 0.01952378  0.08794672 -0.00983255 ...  0.04430017 -0.07930883
    0.0444981 ]
  [-0.01859622 -0.05143956 -0.06700809 ...  0.010121   -0.0286046
   -0.05734096]
  ...
  [-0.05011694  0.08165007 -0.01309595 ...  0.0053298   0.03727864
   -0.01846884]
  [ 0.02138292  0.06931078  0.07190061 ... -0.0333699   0.0184402
    0.01903159]
  [-0.04757103 -0.05319236 -0.02576677 ...  0.02607901  0.01257281
   -0.03387826]]>,
 <Variable path=mlp_block/linear_3/variable_5, shape=(32,), dtype=float32, value=[ 0.0097556   0.05165086 -0.02168501 -0.04811053 -0.01870719 -0.00043184
  -0.0812083   0.05321197 -0.06985502 -0.07663559  0.11270075  0.11589529
   0.061628    0.00150965 -0.02444956  0.00583675 -0.03458467 -0.0378119
  -0.04720056  0.0439657   0.00297305 -0.00412987 -0.05989084  0.07803579
  -0.06123942  0.02284733  0.03370844

Let's connect [YouTube](http://youtube.com/tirendazacademy) | [Medium](http://tirendazacademy.medium.com) | [Twitter](http://twitter.com/tirendazacademy) | [Instagram](https://www.instagram.com/tirendazacademy) | [GitHub](http://github.com/tirendazacademy) | [Linkedin](https://www.linkedin.com/in/tirendaz-academy) | [Kaggle](https://www.kaggle.com/tirendazacademy) 😎